## Importing, Dowloading, and Reading in Forest Watch Landcover data ##

### Part 1: loading in data... 

Goal: Import raster data

https://www.earthdatascience.org/courses/scientists-guide-to-plotting-data-in-python/plot-spatial-data/customize-raster-plots/plotting-extents/

In [5]:
#based on above article importing possibly useful packages
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas as gpd
import rasterio as rio
from rasterio.plot import plotting_extent

#currenly no package calles earthpy (will need to install?)
#import earthpy as et
#import earthpy.spatial as es
#import earthpy.plot as ep

In [6]:
#Issue: loss file is too large to host in the latis platform....try in colab?
loss = rio.open("Hansen_GFC-2020-v1.8_lossyear_40N_010W.tif")

### Section 2: making a basemap and overlaying raster data (tree loss)

In [4]:
#based on lab 7: using folium package
import folium
from folium import features

In [42]:
loss.bounds

BoundingBox(left=-10.0, bottom=30.0, right=0.0, top=40.0)

In [ ]:
#viewing raster data
plt.imshow(loss.read(1))
plt.show()

### Simple summary of stats in this file

attempting to make a histogram that shows the amount of pixels designated to each 'class' (year of tree loss)


In [7]:
#error: 'DataReader' object is not iterable!!!
count0 = 0
count1 = 0
for col in loss: 
    for row in col:
        if row == 1:
            count1 +=1
print(count1)

TypeError: 'DatasetReader' object is not iterable

In [31]:
#chosen basemap and zoomed appropriatley... to current GFW map extant  
map1 = folium.Map(location=[39, 0], zoom_start=5)

GFW_line1 = features.ColorLine(
    positions=list([[30, 0],[40, 0]]),
    colors=[3],
    colormap=['r', 'red'],
    weight=1)

GFW_line2 = features.ColorLine(
    positions=list([[30,-10],[40,-10]]),
    colors=[4],
    colormap=['r','red'],
    weight=1)

GFW_line3 = features.ColorLine(
    positions=list([[40,-10],[40,0]]),
    colors=[4],
    colormap=['r','red'],
    weight=1)

GFW_line4 = features.ColorLine(
    positions=list([[30,-10],[30,0]]),
    colors=[4],
    colormap=['r','red'],
    weight=1)

# Add to map (another way)
GFW_line1.add_to(map1)
GFW_line2.add_to(map1)
GFW_line3.add_to(map1)
GFW_line4.add_to(map1)

map1

### trying to follow a GITHUB demonstartion for ImageOverlay using folium: 
https://github.com/GISerDaiShaoqing/My-Studies-of-Urban-GIS/blob/master/3.Spatial%20visualization%20demo%20in%20folium(for%20Python)/src/foliumrastervisdemo.py

In [18]:
#(entirely copied code...)
#Load library
import gdal
import folium
from folium import plugins

#Open raster file
#driver=gdal.GetDriverByName('/Hansen_GFC-2020-v1.8_lossyear_40N_010W.tif')
#driver.Register() 
ds = gdal.Open('Hansen_GFC-2020-v1.8_lossyear_40N_010W.tif') 
if ds is None:
    print('Could not open')

#Get coordinates, cols and rows
geotransform = ds.GetGeoTransform()
cols = ds.RasterXSize 
rows = ds.RasterYSize 

#Get extent
xmin=geotransform[0]
ymax=geotransform[3]
xmax=xmin+cols*geotransform[1]
ymin=ymax+rows*geotransform[5]

#Get Central point
centerx=(xmin+xmax)/2
centery=(ymin+ymax)/2

#Raster convert to array in numpy
bands = ds.RasterCount
band=ds.GetRasterBand(1)
dataset= band.ReadAsArray(0,0,cols,rows)
dataimage=dataset
dataimage[dataimage[:,:]==-340282346638528859811704183484516925440.000]=0


### problems arise in mapping...

In [19]:
#Visualization in folium
map= folium.Map(location=[centery, centerx], zoom_start=7,tiles='Stamen Terrain')
plugins.ImageOverlay(
    image=dataimage,
    bounds=[[ymin, xmin], [ymax, xmax]],
    colormap=lambda x: (1, 0, x, x),#R,G,B,alpha
).add_to(map)

#Save html
#map.save('wd.html')

AttributeError: module 'folium.plugins' has no attribute 'ImageOverlay'

### trying to avoid plugins...
https://github.com/python-visualization/folium/blob/master/examples/ImageOverlay.ipynb

In [39]:
#i believe that mapping raster imaes is differnt from vector, and that folium only like to made vector like data

#so, i tried to convert my imported raster image into json...FAiled of course
ds = ds.to_json()

AttributeError: 'Dataset' object has no attribute 'to_json'

In [23]:
m = folium.Map([37, 0], zoom_start=3)

folium.PolyLine([[0, 30], [10, 40]]).add_to(m)

folium.raster_layers.ImageOverlay(
    image=ds,
    bounds=[[0, -60], [60, 60]],
    origin="lower",
    colormap=lambda x: (1, 0, 0, x),
    #mercator_project=True,
).add_to(m)

m

TypeError: Object of type Dataset is not JSON serializable

### Simplest thought, just add raster data to map

In [43]:
#based on the error:
#    "'DatasetReader' object has no attribute 'add_to'" I believe i need to read in the raster data through another way?
loss.add_to(map1)

map1

AttributeError: 'DatasetReader' object has no attribute 'add_to'